# Chatbot Rumah sakit terintegrasi telegram

In [1]:
import pandas as pd
import numpy as np
import nltk
import string
import warnings
import requests
import random
from tokens import token
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

warnings.filterwarnings("ignore")

nltk.download('wordnet')
nltk.download('punkt')

from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\naufa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\naufa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Data latih

In [2]:
data = [["Hai",0],["Halo",0],["Hey",0],["Assalamualaikum",0],["Salam",0],
        ["Apakah ada kamar tersedia?",1],["Apa ada kamar kosong?",1],["Apakah ada UGD yang kosong?",1],["Apa ada ruangan kosong?",1],["Apakah ada kamar kosong saat ini?",1],["Apakah ada ruangan kosong sekarang?",1],
        ["Apakah ada dokter?",2],["Apakah ada tenaga medis?",2], ["Apa ada dokter hari ini?",2],["Ada berapa dokter?",2],["Dokter apa saja yang ada?",2],["Apakah ada dokter yang tersedia?",2],   
        ["Disini ada dokter spesialis apa saja?",3],["Di rumah sakit ini ada dokter spesialis apa saja?",3],["Apakah disini ada dokter spesialis?",3],["Dokter spesialis apa saja yang ada disini?",3],
        ["Ruangan apa saja yang ada disini?",4],["Di rumah sakit ini ada ruangan apa saja?",4], ["Berapa jumlah ruangan di rumah sakit ini?",4],["Berapa banyak ruangan di rs ini?",4],["Ruangan apa saja yang tersedia disini?",4],["Ada ruangan apa saja diini?",4],
        ["Apakah bisa menggunakan BPJS?",5],["Apa bisa pakai BPJS?",5],["Apakah bisa pakai BPJS di rumah sakit ini?",5],["Apa di rumah sakit ini menyediakan layanan BPJS?",5],["Bagaimana dengan BPJS?",5],["Apakah bisa memakai BPJS?",5],
        ["Apa saja gejala covid-19?",6],["Seperti apa gejala covid?",6],["Gejala corona seperti apa?",6],["Apakah covid berbahaya?",6],["Apa gejala orang yang terpapar covid19?",6],
        [" ",7],[" ",7],[" ",7],[" ",7],[" ",7],[" ",7],[" ",7],[" ",7],[" ",7],[" ",7],
        ["Apakah disini menerima pasien covid-19?",8],["Apa disini menerima rujukan covid?",8],["Apa disini menerima pasien corona?",8],["Apakah pasien covid bisa dirawat di sini?",8],["Apakah RS ini menerima pasien Covid-19?",8],["Apakah pasien covid boleh dirawat di sini?",8],
        ["Jadwal besuk",9],["Jadwal besuk pasien",9], ["Bertemu pasien",9],["Menjenguk pasien",9],["Saya ingin bertemu pasien",9],["Saya ingin membesuk pasien",9],
        ["Jadwal praktik dokter",10],["Jadwal praktek dokter",10],["Dokter apa saja yang praktik hari ini?",10],["Praktek dokter hari ini",10],["Praktik dokter hari ini",10],
        ["Apakah bisa membeli obat disini?",11],["Apa disini sedia obat?",11],["Apakah tersedia obat disini sedia?",11],["Apa bisa beli obat disini?",11],["Apakah tersedia obat?",11],
        ["Selamat tinggal",12],["Sampai jumpa",12],["Dadah",12],["Bye",12],["Sampai jumpa lagi",12],["Sampai jumpa lain waktu",12],
        ["Terima kasih",13],["Terimakasih",13],["Makasih",13],["Terima kasih banyak",13],["Makasih banyak",13],["Terima kasih banget",13],["Trims",13],["Makasi",13],["Terimakasi",13],
        ["Apa yang bisa kamu lakukan?",14],["Apa saja yang bisa kamu lakukan",14],["Apa yang anda lakukan?",14],["Apa saja yang anda lakukan?",14],
        ["Aku merasa sakit",15],["Saya merasa sakit",15],["Saya ingin berobat",15],["Aku ingin berobat",15],["Aku sakit",15],["Saya sakit",15]]

In [4]:
#Mengubah data menjadi DataFrame agar daoat dilatih
df = pd.DataFrame(data, columns = ["Text","Intent"])
print(df)

                  Text  Intent
0                  Hai       0
1                 Halo       0
2                  Hey       0
3      Assalamualaikum       0
4                Salam       0
..                 ...     ...
90   Saya merasa sakit      15
91  Saya ingin berobat      15
92   Aku ingin berobat      15
93           Aku sakit      15
94          Saya sakit      15

[95 rows x 2 columns]


## Text processing

In [5]:
# Lemmatisasi menggunakan WordNet

lemmer = nltk.stem.WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def Normalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

# Membuat dan melatih model

In [6]:
x = df['Text']
y = df['Intent']

In [7]:
#Deklarasi stopword
stop_factory = StopWordRemoverFactory()
stopwords = stop_factory.get_stop_words()
stopword = stop_factory.create_stop_word_remover()

In [8]:
vectorizer = TfidfVectorizer(tokenizer=Normalize, stop_words = stopwords)

In [9]:
X = vectorizer.fit_transform(x)

In [10]:
# Latih data dengan Logistic Regression
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

lr.fit(X, y)

LogisticRegression()

In [11]:
# Test dengan kalimat
X_test = ["ada ruangan kosong ga ya?"]
prediction = lr.predict(vectorizer.transform(X_test))

In [12]:
prediction

array([1], dtype=int64)

In [13]:
# Untuk mendapatkan probabilitas data uji yang ada di setiap Intent

a = lr.predict_proba(vectorizer.transform(X_test))
a

array([[0.04151427, 0.30735138, 0.0343903 , 0.02206133, 0.13629717,
        0.03703762, 0.03216492, 0.08685247, 0.03501044, 0.03940368,
        0.03038876, 0.02964387, 0.04451617, 0.06183101, 0.02376307,
        0.03777354]])

# Membuat respon untuk masing" intent

In [14]:
responses = {
      0 : {"intent":"salam","response":['Hai','Halo', 'Senang bertemu Anda',]}, 
      1 : {"intent":"ruang_kosong","response":['Saat ini tersedia ruang:','Untuk saat ini ada ruang:','Saat ini hanya ada ruang:', 'Untuk saat ini tersedia ruang:',]},
      2 : {"intent":"dokter","response":['Untuk saat ini ada dokter:', 'Dokter yang ada untuk saat ini adalah:','Saat ini tersedia dokter:']},
      3 : {"intent":"dokter_spesialis","response":['Dokter spesialis yang ada di RS ini antara lain:','Di RS ini ada beberapa dokter spesialis, yaitu:','Beberapa dokter spesialis ada disini, yaitu:',]},
      4 : {"intent":"fasilitas","response":['Di RS ini terdapat beberapa ruangan, antara lain:','Ruangan yang ada di RS ini diantaranya:','RS ini memiliki beberapa ruangan, yaitu:']},
      5 : {"intent":"bpjs","response":['disini bisa menggunakan layanan BPJS','di rumah sakit ini bisa menggunakan BPJS','layanan BPJS tersedia di rumah sakit ini']},
      6 : {"intent":"gejala_covid","response":['Untuk gejala Covid-19 bisa dilihat di https://www.halodoc.com/artikel/tanda-tanda-long-covid-19-yang-perlu-diketahui']},
      7 : {"intent":"tidak_mengerti","response":['Maaf, saya tidak mengerti']},
      8 : {"intent":"rujukan_covid","response":['disini kami menerima pasien dengan gejala Covid-19','di RS ini menerima pasien dengan gejala Covid-19','pasien dengan gejala Covid bisa dirawat disini']},
      9 : {"intent":"Jadwal_besuk","response":['Jadwal besuk adalah pukul','Anda bisa menjenguk pasien pada jam','Jika ingin menemui pasien, harap datang jam']},
      10 : {"intent":"jadwal_praktek","response":['Jadwal praktek dokter di RS ini:','Berikut jadwal praktek dokter disini:','Jadwal praktek harian di RS ini:',]},
      11 : {"intent":"obat","response":['di RS ini tersedia obat','disini anda bisa menebus obat','disini anda bisa membeli obat',]},
      12 : {"intent":"selamat_tinggal","response":['Sampai jumpa','Selamat tinggal','dadah']},
      13 : {"intent":"terimakasih","response":['Sama-sama','Tidak masalah','Terima kasih kembali','Jangan sungkan']},
      14 : {"intent":"info","response":['Yang bisa saya lakukan, antara lain:']},
      15 : {"intent":"sakit","response":['silahkan hubungi dokter dan kunjungi RS terdekat atau RS kami yang berada di lokasi berikut:']}}

# Mengaktifkan bot Telegram

In [15]:
import json

class telegram_bot():
    def __init__(self):
        self.token= token
        self.url = f"https://api.telegram.org/bot{self.token}"

    def get_updates(self,offset=None):
        url = self.url+"/getUpdates?timeout=100"
        if offset:
            url = url+f"&offset={offset+1}"
        url_info = requests.get(url)
        return json.loads(url_info.content)
    
    def send_message(self,msg,chat_id):
        url = self.url + f"/sendMessage?chat_id={chat_id}&text={msg}"
        if msg is not None:
            requests.get(url)

    def grab_token(self):
        return tokens

## Deploy bot dan membuat fungsi reply

In [16]:
def intent(user_response):
    text_intent = [user_response]
    X_test_intent = vectorizer.transform(text_intent)
    predicted_intent = lr.predict(X_test_intent)
    intent_predicted = responses[predicted_intent[0]]['intent']
    return intent_predicted

In [17]:
import random

def bot_initialize(user_msg):
    flag=True
    while(flag==True):
        user_response = user_msg
        
        user_intent = intent(user_response)
        
        if(user_intent != 'selamat_tinggal'):
            if(user_response == '/start'):
                resp = """Salam kenal, saya Chatbot rumah sakit. Saya berharap saya dapat membantu Anda"""
                return resp

            elif (user_intent == 'info' or user_response == '/help'):
                resp = str(random.choice(responses[14]['response'])) +  '''
1. /start
2. /help
3. menyapa
4. info ruangan kosong
5. info dokter tersedia
6. info dokter spesialis
7. info fasilitas
8. info BPJS
9. info gejala Covid-19
10. info rujukan covid
11. info jadwal jenguk
12. info jadwal praktek
13. info membeli obat
14. ucapan selamat tinggal
15. balasan terima kasih
16. Error handling'''
                return resp
            
            elif (user_intent == 'salam'):
                resp = str(random.choice(responses[0]['response'])) + ", Ada yang bisa saya bantu?"
                return resp
        
            elif(user_intent == 'terimakasih'):
                resp = random.choice(responses[13]['response'])
                return resp
            
            elif(user_intent == 'ruang_kosong'):
                resp = random.choice(responses[1]['response']) + "\n 5 Unit Gawat Darurat \n 7 Instalasi Gawat Darurat \n 3 Ruang operasi \n 10 Ruang rawat inap \n 6 Ruang isolasi pasien Covid-19"
                return resp
            
            elif(user_intent == 'dokter'):
                resp = random.choice(responses[2]['response']) + "\n 4 Dokter Umum \n 1 Dokter Spesialis Jantung \n 1 Dokter Spesalis Syaraf"
                return resp
            
            elif(user_intent == 'dokter_spesialis'):
                resp = random.choice(responses[3]['response']) + "\n 1. Dokter Budi (Spesialis Jantung) \n 2. Dokter Ani (Spesialis Gigi) \n 3. Dokter Joko (Spesialis Mata) \n 4. Dokter Amel (Spesialis Syaraf) \n DSB"
                return resp
            
            elif(user_intent == 'fasilitas'):
                resp = random.choice(responses[4]['response']) + "\n 10 Unit Gawat Darurat \n 15 Instalasi Gawat Darurat \n 5 Ruang operasi \n 20 Ruang rawat inap \n 15 Ruang isolasi pasien Covid-19"
                return resp
            
            elif(user_intent == 'bpjs'):
                resp = "Ya, " + random.choice(responses[5]['response'])
                return resp
            
            elif(user_intent == 'gejala_covid'):
                resp = random.choice(responses[6]['response']) + ", \n \nINGAT! Jangan pernah diagnosa penyakit sendiri atau bukan dengan ahlinya, segera hubugi dokter jika memiliki gejala tersebut"
                return resp
 
            elif(user_intent == 'tidak_mengerti'):
                resp = random.choice(responses[7]['response'])
                return resp
            
            elif(user_intent == "rujukan_covid"):
                resp = "Ya, " + random.choice(responses[8]['response'])
                return resp

            elif(user_intent == 'Jadwal_besuk'):
                resp = random.choice(responses[9]['response']) + "13:00 - 17:30, Hari Senin - Sabtu"
                return resp
            
            elif(user_intent == 'jadwal_praktek'):
                resp = random.choice(responses[10]['response']) + "\n Dokter Umum: Senin - Minggu (08:00 - 19-00) \n Dokter Spesialis Mata: Selasa - Jumat (10:00 - 18-30) \n Dokter Spesialis Jantung: Senin - Jumat (10:00 - 17-30) \n DSB"
                return resp
            
            elif(user_intent == 'obat'):
                resp = "Ya, " + random.choice(responses[11]['response']) + ", silahkan tebus obat di apotek Rumah Sakit."
                return resp
            
            elif(user_intent == 'sakit'):
                resp = "Jika merasa sakit dan ingin berobat " + random.choice(responses[15]['response']) + " https://goo.gl/maps/DQFF5JvcFthAxeXn7"
                return resp

            else:
                resp = "Maaf saya tidak mengerti, coba olah kata kembali"
                return resp
            
        else:
            flag = False
            resp = random.choice(responses[12]['response']) + ", semoga sehat selalu :)"
            return resp

In [ ]:
tbot = telegram_bot()
update_id = None

def make_reply(msg):
    if msg is not None:
        reply = bot_initialize(msg)
    return reply

while True:
    print("siap menerima pesan")
    updates = tbot.get_updates(offset=update_id)
    updates = updates['result']
    print(updates)
    if updates:
        for item in updates:
            update_id = item["update_id"]
            print(update_id)
            try:
                message = item["message"]["text"]
                print(message)
            except:
                message = None
            from_ = item["message"]["from"]["id"]
            print(from_)

            reply = make_reply(message)
            tbot.send_message(reply,from_)

siap menerima pesan
[{'update_id': 28502744, 'message': {'message_id': 206, 'from': {'id': 1007745568, 'is_bot': False, 'first_name': 'Naufaldi', 'last_name': 'Hafid', 'username': 'naufaldi_h', 'language_code': 'en'}, 'chat': {'id': 1007745568, 'first_name': 'Naufaldi', 'last_name': 'Hafid', 'username': 'naufaldi_h', 'type': 'private'}, 'date': 1640158677, 'text': '/start', 'entities': [{'offset': 0, 'length': 6, 'type': 'bot_command'}]}}]
28502744
/start
1007745568
siap menerima pesan
[{'update_id': 28502745, 'message': {'message_id': 208, 'from': {'id': 1007745568, 'is_bot': False, 'first_name': 'Naufaldi', 'last_name': 'Hafid', 'username': 'naufaldi_h', 'language_code': 'en'}, 'chat': {'id': 1007745568, 'first_name': 'Naufaldi', 'last_name': 'Hafid', 'username': 'naufaldi_h', 'type': 'private'}, 'date': 1640158689, 'text': '/help', 'entities': [{'offset': 0, 'length': 5, 'type': 'bot_command'}]}}]
28502745
/help
1007745568
siap menerima pesan
[{'update_id': 28502746, 'message': {'me